**Лабораторная работа №1**



Выполнили: Баринов Даниил, Бугров Лев, Голованов Денис

In [1]:
import random
import time
from collections import deque
from tabulate import tabulate

Наивный алгоритм

In [2]:
def simple_search(text, template):
    n = len(text)
    m = len(template)
    comp_count = 0
    for i in range(n - m + 1):
        comp_count += m
        if text[i: i + m] == template:
            return i, comp_count
    return -1

Алгоритм Рабина-Карпа

In [3]:
def rabin_karp_search(text, template, d = 10, q = 17):
    n = len(text)
    m = len(template)
    comp_count = 0
    h = d **(m - 1) % q
    curr_text_hash = 0
    template_hash = 0
    
    for i in range(m):
        curr_text_hash = (d * curr_text_hash + ord(template[i])) % q
        template_hash = (d * template_hash + ord(text[i])) % q
    
    for i in range(n - m + 1):
        comp_count += 1
        if curr_text_hash == template_hash:
            comp_count += m
            if template == text[i:i + m]:
                return i, comp_count
        
        if i < n-m:
            template_hash = (template_hash - h * ord(text[i])) % q 
            template_hash = (template_hash * d + ord(text[i + m])) % q
            template_hash = (template_hash + q) % q
    return -1

Алгоритм Кнута-Морис-Пратта

In [4]:
def prefix(template):
    S = [0]*len(template)
    for i in range(1,len(template)):
        k = S[i-1]
        while k > 0 and template[k] != template[i]:
            k = S[k-1]
        if template[k] == template[i]:
            k = k + 1
        S[i] = k
    return S


def kmp_search(text,template):
    comp_count = 0
    p = prefix(template)
    i = j = 0
    while i < len(text) and j < len(template):
        comp_count += 1
        if template[j] == text[i]:
            i+=1
            j+=1
        elif j==0:
            i+=1
        else:
            j = p[j-1]

    if j == len(template):
        return i-j, comp_count
    return -1

Тесты и проверки на бенчмарках

In [5]:
all_benchmark_paths = [
    'benchmarks/bad_t_1.txt',
    'benchmarks/bad_t_2.txt',
    'benchmarks/bad_t_3.txt',
    'benchmarks/bad_t_4.txt',
    'benchmarks/bad_w_1.txt',
    'benchmarks/bad_w_2.txt',
    'benchmarks/bad_w_3.txt',
    'benchmarks/bad_w_4.txt',
    'benchmarks/good_t_1.txt',
    'benchmarks/good_t_2.txt',
    'benchmarks/good_t_3.txt',
    'benchmarks/good_t_4.txt',
    'benchmarks/good_w_1.txt',
    'benchmarks/good_w_2.txt',
    'benchmarks/good_w_3.txt',
    'benchmarks/good_w_4.txt',
]

def read_benchmark_files(benchmark_paths):
    benchmarks = dict()
    for benchmark_path in benchmark_paths:
        with open(benchmark_path, 'r', encoding='utf-8') as f:
            benchmarks[benchmark_path[11:]] = f.read()
    return benchmarks

benchmarks = read_benchmark_files(all_benchmark_paths)

In [6]:
def unit_test_search(text, search_algorithm, gap):
    print(f'text: {text}')
    n = len(text)
    idx = random.randint(0, n - 1)
    template = text[idx:idx + gap % n]
    print(f'template: {template}')
    print(f'test: {search_algorithm(text, template)}, true: {text.find(template)}')
    assert search_algorithm(text, template)[0] == text.find(template)
    print("Unit test succesfully passed")

In [7]:
def stress_test_search(text, search_algorithm, n_iter):
    n = len(text)
    for i in range(n_iter):
        idx = random.randint(0, n - 1)
        gap = random.randint(0, n - 1)
        template = text[idx:idx + gap % n]
        assert search_algorithm(text, template)[0] == text.find(template)
    print("Stress test successfully passed")

In [8]:
def measure_algorithm_time_and_validate(search_algorithm, text, template):
    n = len(text)
    start_time = time.time()
    result = search_algorithm(text, template)
    end_time = time.time()
    assert result[0] == text.find(template)
    return end_time - start_time, result[1]

In [9]:
def compute_results(benchmarks):
    results = dict()
    templates_keys = []
    for key in benchmarks:
        if '_t' in key:
            results[key] = []
        else:
            templates_keys.append(key)
            
    i = 0
    for key in results:
        text = benchmarks[key]
        template = benchmarks[templates_keys[i]]
        i += 1
        
        results[key].append(measure_algorithm_time_and_validate(simple_search, text, template))
        results[key].append(measure_algorithm_time_and_validate(rabin_karp_search, text, template))
        results[key].append(measure_algorithm_time_and_validate(kmp_search,text, template))
    return results

In [10]:
def display_table_of_results(results, benchmarks):
    table_time = [[' ', 'Naive', 'Rabin-Karp', 'Knuth-Morris-Pratt']]
    
    for benchmark in results:
        table_time.append([benchmark] + list(map(lambda x: f'{round(x[0], 5)} / {x[1]}', results[benchmark])))
    
    print('Measurments of running time of algorithms and the number of comparisons (time, s / comparions number)')
    print(tabulate(table_time, tablefmt='fancy_grid'))

In [11]:
unit_test_search(benchmarks['good_w_2.txt'], simple_search, 10)
stress_test_search(benchmarks['good_w_4.txt'], simple_search, 1000)

text: Да, я слышал про его план вечного мира, и это очень интересно, но едва ли возможно...
template: о очень ин
test: (44, 450), true: 44
Unit test succesfully passed
Stress test successfully passed


In [12]:
unit_test_search(benchmarks['good_w_4.txt'], rabin_karp_search, 100)
stress_test_search(benchmarks['good_w_1.txt'], rabin_karp_search, 1000)

text: Бутылка рому была принесена; раму, не пускавшую сесть на наружный откос окна, выламывали два
template: ринесена
test: (19, 44), true: 19
Unit test succesfully passed
Stress test successfully passed


In [13]:
unit_test_search(benchmarks['good_w_3.txt'], kmp_search, 100)
stress_test_search(benchmarks['good_w_3.txt'], kmp_search, 1000)

text: вспоминал свое обещание, но тут же, как это бывает с людьми, называемыми бесхарактерными, ему так страстно захотелось войти взглянуть еще раз на эту столь знакомую и надоевшую ему беспутную жизнь, и невольно пришла в голову мысль, что данное слово ничего не значит, к тому же, еще прежде, чем князю Андрею, он дал также Анатолю слово привезти долг; наконец, он подумал, что все эти
template: характерными, ему так страстно захотелось войти взглянуть еще раз на эту столь знакомую и надоевшую 
test: (76, 176), true: 76
Unit test succesfully passed
Stress test successfully passed


Результаты

In [14]:
results = compute_results(benchmarks)

In [15]:
display_table_of_results(results, benchmarks)

Measurments of running time of algorithms and the number of comparisons (time, s / comparions number)
╒══════════════╤═════════════════╤═════════════════╤════════════════════╕
│              │ Naive           │ Rabin-Karp      │ Knuth-Morris-Pratt │
├──────────────┼─────────────────┼─────────────────┼────────────────────┤
│ bad_t_1.txt  │ 0.0 / 18        │ 0.0 / 11        │ 0.0 / 18           │
├──────────────┼─────────────────┼─────────────────┼────────────────────┤
│ bad_t_2.txt  │ 0.0 / 910       │ 0.0 / 101       │ 0.0 / 190          │
├──────────────┼─────────────────┼─────────────────┼────────────────────┤
│ bad_t_3.txt  │ 0.0 / 90100     │ 0.00099 / 1001  │ 0.0 / 1900         │
├──────────────┼─────────────────┼─────────────────┼────────────────────┤
│ bad_t_4.txt  │ 0.002 / 4001000 │ 0.002 / 5001    │ 0.00363 / 9000     │
├──────────────┼─────────────────┼─────────────────┼────────────────────┤
│ good_t_1.txt │ 0.00099 / 10200 │ 0.0 / 1110      │ 0.0 / 633          │
├─────────